In [1]:
import numpy as np

import getpass
user = getpass.getuser()

In [2]:
from PIL import Image

In [ ]:
from omni.isaac.kit import SimulationApp

# "/home/yizhou/Research/OpenAnyDrawer/scene0.usd" # 
usd_path = f"omniverse://localhost/Users/{user}/scene1.usd"


simulation_app = SimulationApp({"headless": True, "open_usd": usd_path,  "livesync_usd": usd_path}) 


Starting kit application with the fillowing args:  ['/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.0/exts/omni.isaac.kit/omni/isaac/kit/simulation_app.py', '/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.0/apps/omni.isaac.sim.python.kit', '--/app/tokens/exe-path=/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.0/kit', '--/persistent/app/viewport/displayOptions=3094', '--/rtx/materialDb/syncLoads=True', '--/rtx/hydra/materialSyncLoads=True--/omni.kit.plugin/syncUsdLoads=True', '--/app/renderer/resolution/width=1280', '--/app/renderer/resolution/height=720', '--/app/window/width=1440', '--/app/window/height=900', '--/renderer/multiGpu/enabled=True', '--ext-folder', '/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.0/exts', '--ext-folder', '/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.0/apps', '--portable', '--no-window']
Passing the following args to the base kit application:  ['-f', '/home/yizhou/.local/share/jupyter/runtime/kernel-d9dd16b4-ec5c-4ef1-a784-e2da8985388b.json

OmniAssetFileFormat
2022-08-05 19:16:30 [2,261ms] [Error] [rtx.neuraylib.plugin] [DYNLIB:IO]   0.1   DYNLIB io   error: /home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.0/kit/python/bin/libs/iray/libnvindex.so: cannot open shared object file: No such file or directory
2022-08-05 19:16:30 [2,261ms] [Error] [rtx.neuraylib.plugin] [INDEX:MAIN]   0.1   INDEX  main error: Failed to load /home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.0/kit/python/bin/libs/iray/libnvindex.so


[1.897s] [ext: omni.mdl-0.1.0] startup
[1.920s] [ext: omni.kit.menu.create-1.0.2] startup
[1.922s] [ext: omni.kit.window.file_exporter-1.0.4] startup
[1.924s] [ext: omni.kit.window.drop_support-1.0.0] startup
[1.925s] [ext: omni.kit.material.library-1.3.10] startup
[1.929s] [ext: omni.kit.window.property-1.6.3] startup
[1.931s] [ext: omni.kit.context_menu-1.3.9] startup
[1.935s] [ext: omni.kit.window.file_importer-1.0.4] startup
[1.937s] [ext: omni.kit.stage_templates-1.1.2] startup
[1.940s] [ext: omni.kit.widget.stage-2.6.15] startup
[1.945s] [ext: omni.kit.window.file-1.3.16] startup
[1.949s] [ext: omni.debugdraw-0.1.0] startup
[1.956s] [ext: omni.kit.window.content_browser-2.4.28] startup
[1.969s] [ext: omni.kit.widget.prompt-1.0.1] startup
[1.970s] [ext: omni.kit.property.usd-3.14.8] startup
[2.011s] [ext: omni.hydra.engine.stats-1.0.0] startup
[2.018s] [ext: omni.kit.widget.settings-1.0.0] startup
[2.021s] [ext: omni.graph.tools-1.3.5] startup
[2.099s] [ext: omni.graph.core-2.27.0

In [ ]:
# world
from omni.isaac.core import World
world = World()

In [ ]:
# reset scene
import omni
mobility_prim = world.scene.stage.GetPrimAtPath("/World/Game/mobility")
if mobility_prim:
    omni.kit.commands.execute("DeletePrims", paths=["/World/Game/mobility"])

In [ ]:
from open_env import OpenEnv

env = OpenEnv()

In [ ]:
from hand_env import HandEnv

# env = HandEnv("/World/Hand/Bones/l_carpal_mid", "/World/Hand*/Bones/l_thumbSkeleton_grp/l_distalThumb_mid")
controller = HandEnv("/World/allegro*/allegro_mount", "/World/AnchorXform")

In [ ]:
env.setup_viewport()

In [ ]:
world.render()
env.get_image()

In [ ]:
env.add_object(0, scale = 0.1)

In [ ]:
from omni.isaac.core.prims.xform_prim import XFormPrim
mobility_obj = XFormPrim("/World/Game/mobility")
mobility_obj_name = mobility_obj.name

world.scene.add(mobility_obj)

In [ ]:
# reset scene
replicator_prim = world.scene.stage.GetPrimAtPath("/Replicator")
if replicator_prim:
    omni.kit.commands.execute("DeletePrims", paths=["/Replicator"])

In [ ]:
# init
world.reset()
controller.start()
world.scene.add(controller.robots)

world.render()

env.get_image()

In [ ]:
from task.checker import TaskChecker
from task.instructor import SceneInstructor

In [ ]:
scene_instr = SceneInstructor()
scene_instr.analysis()
# scene_instr.build_handle_desc_ui()

In [ ]:
scene_instr.is_obj_valid

In [ ]:
scene_instr.valid_handle_list

In [ ]:
handle_path_str = list(scene_instr.valid_handle_list.keys())[0]

handle_joint_type = scene_instr.valid_handle_list[handle_path_str]["joint_type"]
handle_joint = scene_instr.valid_handle_list[handle_path_str]["joint"]
handle_rel_direciton = scene_instr.valid_handle_list[handle_path_str]["relative_to_game_center"]
handle_direction = scene_instr.valid_handle_list[handle_path_str]["direction"]

In [ ]:
handle_path_str, handle_joint_type, handle_joint, handle_rel_direciton

In [ ]:
task_checker = TaskChecker("mobility", handle_joint, handle_joint_type, IS_RUNTIME=True)

# Solve

In [ ]:
world.reset()

In [ ]:
controller.xforms.set_world_poses(positions=np.array([[0,0,0]]), orientations = np.array([[1, 0, 0, 0]])) # WXYZ

for _ in range(60):
    world.step(render=True)

In [ ]:
# get grasp location, if handle is horizontal, gripper should be vertical
graps_pos, grasp_rot = controller.calculate_grasp_location(keyword = handle_path_str, 
                                                         verticle = handle_direction == "horizontal")

In [ ]:
graps_pos, grasp_rot

# Deep Learning

In [ ]:
# scene_instr.load_model()

In [ ]:
# scene_instr.predict_bounding_boxes(image_path="/home/yizhou/Research/temp0/rgb_0.png")

In [ ]:
the_box = [-0.24306170648464168, 0.9585027645051194, -0.2043927986348123, 0.9971716723549489]

In [ ]:
pred_handle_direction = "horizontal" if (the_box[2] - the_box[0]) > (the_box[3] - the_box[1]) else "vertical" 

In [ ]:
pred_handle_direction

In [ ]:
the_box[3] - the_box[1]

In [ ]:
pred_handle_direction

In [ ]:
controller.calculate_grasp_location_from_pred_box(the_box, False)

In [ ]:
graps_pos[...,0] -= 0.1

In [ ]:
controller.xforms.set_world_poses(graps_pos, grasp_rot)

for _ in range(500):
    world.step(render=True)

In [ ]:
graps_pos[...,0] += 0.1

In [ ]:
controller.xforms.set_world_poses(graps_pos, grasp_rot)

for _ in range(100):
    world.step(render=True)

In [ ]:
# close finger

finger_pos = np.array([
    [
     0, 0, 0, np.pi/2 + np.pi/18, 
     np.pi/5, np.pi/5, np.pi/5, 0,
     np.pi/5, np.pi/5, np.pi/5, np.pi/6,
     np.pi/5, np.pi/5, np.pi/5, np.pi/6,
    ],
])

for i in range(120):
    controller.robots.set_joint_position_targets(finger_pos * i / 120) # 
    world.step(render=True)

In [ ]:
# pull out
for i in range(300):
    graps_pos[...,0] -= 0.001
#   env.robots.set_world_poses(graps_pos, grasp_rot)
    controller.xforms.set_world_poses(graps_pos, grasp_rot)
    controller.robots.set_joint_position_targets(finger_pos)
    world.step(render=True)

In [ ]:
# check task
task_checker.joint_checker.compute_percentage()

In [ ]:
world.reset()
controller.xforms.set_world_poses(positions=np.array([[0,0,0]]), orientations = np.array([[1, 0, 0, 0]])) # WXYZ

for _ in range(30):
    world.step()

In [ ]:
world.scene.remove_object(mobility_obj_name)

In [ ]:
world.render()